In [1]:
import csv

import numpy as np
import pandas as pd
import itertools

import pickle
import time

import gensim

import spacy
nlp = spacy.load('en_core_web_sm')

from spacy.lang.en.stop_words import STOP_WORDS
STOP_WORDS.remove('not')

C:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


***

#### FUNCTION THAT RETURN FILE LENGTH WITHOUT LOADING

In [2]:
def retrn_file_lg(filename):
    
    def yld_csv_length(filename):
        
    # returns one for every yield from file
        with open(filename, newline='') as file:
            reader = csv.reader(file)
            for row in reader:
                yield 1
    
    gen1 = yld_csv_length(filename)
    file_lg = 0
    for i in gen1:
        file_lg += 1

    return file_lg

In [4]:
filename = 'df_amazon.csv'

file_lg = retrn_file_lg(filename)
file_lg

364120

#### FUNTION THAT RETURN ROWS FROM CSV ON DISK WITHIN USER SPECIFIED RANGE

In [5]:
def yld_df_rows_csv_on_disk(filename, start, stop, col_idx):    
    
    def yld_csv_rows(filename):

        with open(filename, newline='') as file:
            reader = csv.reader(file)
            for row in reader:
                yield row

    raw_corpus = [] 
    gen_df_rows = yld_csv_rows(filename)
    gen_df_rows = itertools.islice(gen_df_rows, start+1, stop+1)

    for row in gen_df_rows:
        raw_corpus.append(row[col_idx])

    
    
    return raw_corpus

### SAMPLE OF CLEANED TEXT CORPUS

In [6]:
filename  = 'df_amazon.csv'
start, stop = 0, 5
col_idx = 3

raw_corpus = yld_df_rows_csv_on_disk(filename, start, stop, col_idx)

#print(df_amazon_sample.shape)
for i in raw_corpus:
    print(i)
    print('')

have bought several the vitality canned dog food products and have found them all good quality the product looks more like stew than processed meat and smells better labrador finicky and she appreciates this product better than most

product arrived labeled jumbo salted peanuts the peanuts were actually small sized unsalted not sure this was error the vendor intended represent the product jumbo

this confection that has been around few centuries light pillowy citrus gelatin with nuts this case filberts and cut into tiny squares and then liberally coated with powdered sugar and tiny mouthful heaven not too chewy and very flavorful highly recommend this yummy treat you are familiar with the story lewis the lion the witch and the wardrobe this the treat that seduces edmund into selling out his brother and sisters the witch

you are looking for the secret ingredient robitussin believe have found got this addition the root beer extract ordered which was good and made some cherry soda the fl

***

# FUNCTIONS:

# Fn1
- YIELD ROWS BETWEEN SPECIFIED RANGE FROM CSV ON DISK AND 
- FILTER-COLLECT ONLY ADJECTIVES AND IMPORTANT WORDS PER ROW

In [11]:
def retrn2_filtered_corpusN(filename, start, stop, bypass = 1):
    
    from progressbar import ProgressBar    
     
    def yld_csv_rows(filename):
        
        # YIELDS ROWS FROM CSV FILE
        with open(filename, newline='') as file:
            reader = csv.reader(file)
            for row in reader:
                yield row
                
    
    def retrn_only_correct_spelling(token):
        
        # RETURNS NULL IF WORD CONTAINS SEQUENCE OF MORE THAN 2 VOWELS OR 3 CONSONANTS 
        consonants = 'b c d f g h j k l m n p q r s t v w x z'.split()
        vowels = 'a e i o u y'.split()
        c1 = 0
        c2 = 0
        for alpha in token.text:
            if alpha in vowels:
                c1 += 1
                c2 = 0
                if c1 > 2:
                    break
            if alpha in consonants:
                c1 = 0
                c2 += 1
                if c2 > 3:
                    break
        if c1<=2 and c2<=3:
            return token
        
    
    def retrn_final_listO_words(doc):
        
        doc = nlp(doc) #spaCy object
        listO_words = []

        for token in doc:
            
            # adding 'listO_words_to_watch' & some words surronding it.
            if token.text in listO_words_to_watch:
                i = list(doc).index(token)
                for n in [-2, -1, 0, 1, 2]:
                    try:
                        token1 = doc[i+n]
                        condition1 = token1.is_stop is False
                        condition2 = len(token1.lemma_) >= 3
                        condition3 = retrn_only_correct_spelling(token1)
                        
                        if condition1 and condition2 and condition3:
                            listO_words.append(token1.lemma_.lower())
                    except:
                        pass
                    
            # Futher filtering out stopwords & non adjectives 
            condition4 = token.pos_ == 'ADJ'
            condition5 = retrn_only_correct_spelling(token)

            if condition4:  
                for m in [-1, 0, 1]:
                    try:
                        token2 = doc[i+m]
                        if token2.is_stop is false and condition5:
                            listO_words.append(token2.lemma_.lower())
                    except:
                        pass                        
            listO_words = list(set(listO_words))    
        return listO_words
    
    
     # INSTANTIATE GENERATOR:
    gen_df_rows = yld_csv_rows(filename)  
    
    # GENERATE ONLY WITHIN SPECIFIED RANGE:
    gen_df_rows = itertools.islice(gen_df_rows, start+bypass, stop+bypass )
    
    pbar = ProgressBar(max_value=stop-start)
    c_pbar, idx = 0, 0
    filtered_corpus, indexes = [], [] # ListO_listO_words
    
    for row in pbar(gen_df_rows):
        doc = row[3] # 4th column contains cleanedText
        
        listO_words = retrn_final_listO_words(doc)
        filtered_corpus.append(listO_words)
        indexes.append(idx)
        idx += 1
        c_pbar += 1
        pbar.update(c_pbar)
    
    return filtered_corpus, indexes

#  Fn2
- USES Fn1 TO:
    - YIELD ROWS BETWEEN SPECIFIED RANGE FROM CSV ON DISK AND 
    - FILTER-COLLECT ONLY ADJECTIVES AND IMPORTANT WORDS PER ROW
- AND THEN SAVES THE FILTERATE AS MULTIPLE EQUALLY SIZED PICKLED FILES

In [12]:
def fn_filter_corpus_save(filename, start, stop, n_intervals, save_as):
    
    # USES Fn1 TO YIELD ROWS FROM CSV ON DISK AND FILTER-COLLECT ONLY NOUNS AND VERBS PER SENTENCE,
    # AND THEN SAVES FILTERATE AS MULTIPLE EQUALLY SIZED PICKLED FILES
    
    c1 = 0
    c2 =  c1 + 1
    n_items =  n_intervals+1
    z = [int(i) for i in list(np.linspace(start, stop,  n_items))] 

    idx = []
    while True:
        final_corpus, indexes = retrn2_filtered_corpusN(filename, z[c1], z[c2]) # Fn1
        idx += indexes
        file_name = save_as + str(c2)
        np.save(file_name, final_corpus, allow_pickle=True, fix_imports=True) 
        print(c1+1, "processed_files saved")
        
        c1 += 1
        c2 = c1 + 1
        if c1 > n_intervals-1:
            break
            
    return idx

# START

### LOADING RAW_CORPUS, PROCESSING TO NOUN_VERB BASED CORPUS AND SAVING PROCESSED FILES:

In [13]:
listO_words_to_watch = ['not', 'dont', 'yes', 'perfect', 'love', 'loved', 'loves', 'liked', 'likes', 'average', 'inferior', 'expectation', 'expect', 
                        'disappoint', 'avoid', 'worth', 'never', 'worse', 'loath', 'deceive', 'beautifully', 'wrong', 'missing', 
                        'concern', 'price', 'cost', 'money','too', 'very', 'fool', 'fooled', 'appreciate', 'again', 'quality', 'amaze', 'wow', 
                        'class', 'classy', 'niche', 'mistake', 'regret', 'reject', 'yuck', 'yucky', 'ugh', 'lacks','lacking',
                        'shit', 'shitty', 'under', 'mouth', 'smell', 'aroma', 'odour', 'flavor', 'taste', 'feel',  'yummy', 'yum', 'super',
                        'texture', 'unique', 'fake', 'original', 'bad', 'too', 'really', 'down', 'hit', 'right', 'excellent'
                        'favourite', 'crunchy', 'smooth', 'rich', 'soft', 'melt', 'relish', 'verdict', 'stars', 'rate', 
                        'more', 'deceptive', 'deceive', 'back', 'again', 'old','spoilt',
                        'sad', 'unhappy', "was'nt", 'good', 'aweful', 'beware']

In [14]:
filename = 'df_amazon.csv'
save_as = 'corpus_ADJECTIVE_amazon_0_to_150_000_'
start = 0
stop = 150_000
n_intervals = 3


%time idx = fn_filter_corpus_save(filename, start, stop, n_intervals, save_as)

100% (50000 of 50000) |##################| Elapsed Time: 0:31:57 Time:  0:31:57


1 processed_files saved


100% (50000 of 50000) |##################| Elapsed Time: 0:31:05 Time:  0:31:05


2 processed_files saved


100% (50000 of 50000) |##################| Elapsed Time: 0:30:38 Time:  0:30:38


3 processed_files saved
Wall time: 1h 33min 43s


In [15]:
len(idx)

150000

In [16]:
filename = 'df_amazon.csv'
save_as = 'corpus_ADJECTIVE_150_000_to_300_000'
start = 150_000
stop = 300_000
n_intervals = 3


%time idx = fn_filter_corpus_save(filename, start, stop, n_intervals, save_as)

100% (50000 of 50000) |##################| Elapsed Time: 0:26:17 Time:  0:26:17


1 processed_files saved


100% (50000 of 50000) |##################| Elapsed Time: 0:29:13 Time:  0:29:13


2 processed_files saved


100% (50000 of 50000) |##################| Elapsed Time: 0:24:56 Time:  0:24:56


3 processed_files saved
Wall time: 1h 20min 32s


In [17]:
len(idx)

150000

In [18]:
filename = 'df_amazon.csv'
save_as = 'corpus_ADJECTIVE_300_000_to_364120_'
start = 300_000
stop = file_lg
n_intervals = 2


%time idx = fn_filter_corpus_save(filename, start, stop, n_intervals, save_as)

100% (32060 of 32060) |##################| Elapsed Time: 0:16:26 Time:  0:16:26


1 processed_files saved


100% (32060 of 32060) |##################| Elapsed Time: 0:15:56 Time:  0:15:56


2 processed_files saved
Wall time: 32min 28s


In [19]:
len(idx)

64119

### CONVERTING PROCESSSED FILES INTO DATAFRAMES AND SAVING AS CSV FILES

In [ ]:
import numpy as np
import pandas as pd

In [23]:
corpus1 = list(np.load('corpus_ADJECTIVE_amazon_0_to_150_000_1.npy'))
corpus2 = list(np.load('corpus_ADJECTIVE_amazon_0_to_150_000_2.npy'))
corpus3 = list(np.load('corpus_ADJECTIVE_amazon_0_to_150_000_3.npy'))
corpus4 = list(np.load('corpus_ADJECTIVE_150_000_to_300_000_1.npy'))
corpus5 = list(np.load('corpus_ADJECTIVE_150_000_to_300_000_2.npy'))
corpus6 = list(np.load('corpus_ADJECTIVE_150_000_to_300_000_3.npy'))
corpus7 = list(np.load('corpus_ADJECTIVE_300_000_to_364120_1.npy'))
corpus8 = list(np.load('corpus_ADJECTIVE_300_000_to_364120_2.npy'))

corpus = corpus1 + corpus2 + corpus3 + corpus4 + corpus5 + corpus6 + corpus7 + corpus8
len(corpus)

364119

### CORPUS DOCS AS LISTO_WORDS:

In [53]:
filename  = 'df_amazon.csv'
start, stop = 0, len(df_amazon)+1
col_idx = 2

score = yld_df_rows_csv_on_disk(filename, start, stop, col_idx)

len(score)

364119

In [54]:
score[:5]

['positive', 'negative', 'positive', 'negative', 'positive']

In [55]:
df_adjective_listO_words = pd.DataFrame(np.array(corpus))

print(df_adjective_listO_words.shape)

df_adjective_listO_words.columns = ['listO_words']
df_adjective_listO_words = df_adjective_listO_words.assign(score = score)
df_adjective_listO_words.head()

(364119, 1)


,listO_words,score
0,"[good, stew, look, product, like, quality]",positive
1,"[unsalted, sized, sure]",negative
2,"[recommend, chewy, heaven, highly, yummy, mout...",positive
3,"[good, medicinal, soda, flavor]",negative
4,"[price, assortment, yummy, delivery, great, ta...",positive


In [56]:
np.save('df_adjective_listO_words', df_adjective_listO_words.listO_words.values, allow_pickle=True, fix_imports=True)

### CORPUS DOCS AS TEXT:

In [65]:
def retrn2_text_from_listO_words(corpus):
    
    adj_text = []
    anomaly = []
    c = 0
    for listO_words in corpus:
        if len(listO_words) > 0:
            text = ' '.join(listO_words)
            adj_text.append(text)
        else:
            adj_text.append('a')
            anomaly.append(c)
        c += 1  
    return adj_text, anomaly

In [66]:
corpus = corpus

corpus_text, anomaly = retrn2_text_from_listO_words(corpus)
len(corpus_text), len(anomaly)

(364119, 29381)

In [67]:
anomaly[:5]

[11, 20, 25, 81, 88]

In [71]:
df_adjective_text = pd.DataFrame(corpus_text)
df_adjective_text.columns = ['text']
df_adjective_text = df_adjective_text.assign(score = score)#.drop(anomaly)


print(df_adjective_text.shape)
df_adjective_text.head()

(364119, 2)


,text,score
0,good stew look product like quality,positive
1,unsalted sized sure,negative
2,recommend chewy heaven highly yummy mouthful f...,positive
3,good medicinal soda flavor,negative
4,price assortment yummy delivery great taffy wi...,positive


In [72]:
df_adjective_text.to_csv('df_adjective_text.csv')